<a href="https://colab.research.google.com/github/Aananda-giri/Bootcamps/blob/langchain/langchain_prompt_enginnering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

from langchain import PromptTemplate

demo_template = '''I want you to act as a acting financial advisor for people. In an easy way, explain the basics of {financial_concept}.'''

prompt=PromptTemplate(
  input_variables=['financial_concept'],
  template=demo_template
)

print(prompt)

prompt.format(financial_concept='income tax')

print(prompt)

input_variables=['financial_concept'] input_types={} partial_variables={} template='I want you to act as a acting financial advisor for people. In an easy way, explain the basics of {financial_concept}.'
input_variables=['financial_concept'] input_types={} partial_variables={} template='I want you to act as a acting financial advisor for people. In an easy way, explain the basics of {financial_concept}.'


In [ ]:
!pip install -qU langchain-google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.llms import OpenAI
from langchain.chains import LLMChain

os.environ["GOOGLE_API_KEY"] = userdata.get('GEMMA_KEY_JOKELEOPEDIA')

# os.environ["OPENAI_API_KEY"]="<your api key>"
# source: prompt templates example
# llm = OpenAI(temperature=0.7)


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
chain1 = LLMChain(llm=llm, prompt=prompt)

chain1.run('income tax.')





"Alright folks, let's talk about income tax – nobody's favorite topic, but definitely something you need to understand.  Think of it like this:  you earn money, and the government takes a little slice of that pie to pay for things like roads, schools, and parks.  That slice is your income tax.\n\nHere's the basic breakdown:\n\n1. **Income:** This is all the money you earn, including your salary, wages, tips, any side hustle money, and even investment earnings.\n\n2. **Deductions:**  Think of these as discounts on your taxes.  There are certain expenses you can deduct from your total income, which lowers the amount you're taxed on.  Common deductions include things like student loan interest, contributions to retirement accounts (like a 401(k) or IRA), and sometimes even charitable donations.  These deductions can significantly reduce your tax bill!\n\n3. **Taxable Income:** This is what's left after you subtract your deductions from your total income.  This is the amount the government

In [ ]:
chain1.run('GDP.')

"Alright folks, gather 'round! Let's talk about GDP – it sounds fancy, but it's really just a way to measure how well our country's economy is doing. Think of it like a national report card for our money-making activities.\n\nImagine a giant bakery.  Everything they bake and sell in a year, from cookies to cakes, adds to their total sales. GDP is like that, but for a whole country. It's the total value of *everything* produced within our borders in a year – all the goods and services, from haircuts to houses, from apps to apples.\n\n**Here's the simple breakdown:**\n\n* **Goods:**  Think tangible stuff – cars, clothes, furniture, food.\n* **Services:**  Things people do for you – doctor visits, haircuts, concerts, online shopping deliveries.\n\n**Why do we care about GDP?**\n\nWell, if GDP is growing, it generally means the economy is doing well.  Businesses are hiring, people are spending money, and things are looking up. If GDP is shrinking, it could mean trouble – businesses might s

## Few shot prompts
* [reference](https://python.langchain.com/docs/how_to/few_shot_examples_chat/#create-prompt-template)

In [ ]:
!pip install -qU langchain langchain-openai langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00


In [ ]:
import  langchain_google_genai as google_genai

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# os.environ["GOOGLE_API_KEY"] = "<your_google_api_key>"

examples = [
    {"input": "2 🥜 2", "output": "4"},
    {"input": "2 🥜 3", "output": "5"},
    {"input": "2 🥜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
# embeddings = GoogleGenerativeAIEmbeddings()
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


'''
this line gives an error:
ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
model
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
'''
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"input": "horse"})

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3 🥜 3?").to_messages())

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="What's 3 🥜 3?"))

[HumanMessage(content='2 🥜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🥜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]
messages=[HumanMessage(content='2 🥜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🥜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]


In [ ]:
chain = final_prompt | ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

chain.invoke({"input": "What's 3 🦜 3?"})

AIMessage(content='If 🦜 represents multiplication, then 3 🦜 3 = 9.\n\nIf 🦜 represents addition, then 3 🦜 3 = 6.\n\nIf 🦜 represents concatenation, then 3 🦜 3 = 33.\n\n🦜 could also represent a completely different operation.  I need more information to give you a definite answer!  Tell me what 🦜 means.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-201f13ff-bf2a-4a4d-98f6-6e501d8e29c8-0', usage_metadata={'input_tokens': 30, 'output_tokens': 85, 'total_tokens': 115, 'input_token_details': {'cache_read': 0}})

In [ ]:
# ## Language Translation (krish naik openai example)

# from langchain import PromptTemplate
# template = '''In an easy way to translate the following sentence '{sentence}' into {target_language}'''
# language_prompt = PromptTemplate(
# input_variables=["sentence", "target_language"],
# template=template
# )
# language_prompt.format(sentencee="How are you", target_language="hindi")

# chain2 = LLMChain(llm=llm, prompt=language_prompt)
# chain2({'sentence':"Hello how are you", "target_language":"hindi"})


# from langchain import PromptTemplate, FewShotPromptTemplate

# # Frist, create the list of feww shot examples
# examples=[
# {"word":"happy", "antonym":"sad"},
# {"word":"tall", "antonym":"short"},
# ]

# # Next, we specify thhe template to format the examples we have provided.
# # We usse the `promptTemplate` class for this
# example_formatter_template = """Word: {word}
# Antonym: {antonym}
# """
# # finally, we create the `FewShotPromptTemplate` object.
# few_shot_prompt = FewShotPromptTemplate(
# # These are the examples we want to inesrt into the prompt.
# examples=examples,
# # Thus us how we want to format the examples when we insert them into the prompt
# example_prompt = example_prompt,
# # The prefix is some text that goes before the examples in the prompt
# # usually, this is where the user input will go
# # The input variables are the variables that ahte overall prompt expects.
# prefix="Give the eantonym of every inpyt\n",
# # The example_seperator
# suffix="Word: {input}\n Antonym: ",

# input_variables=["input"],
# # The example seperator is the string we will use to join the prefix, examples,
# example_separator="\n",
# )

# print(few_shot_prompt.format(input='big'))

# chain = LLMChain(llm=llm, prompt=few_shot_prompt)
# chain.run("big")

input_variables=['sentence', 'target_language'] input_types={} partial_variables={} template='In an easy way to translate the following sentence {sentence} into {target_language}'


In [ ]:
chain2 = LLMChain(llm=llm, prompt=language_prompt)
chain2({'sentence':"Hello how are you", "target_language":"hindi"})


from langchain import PromptTemplate, FewShotPromptTemplate
# Frist, create the list of feww shot examples
examples=[
  {"word":"happy", "antonym":"sad"},
  {"word":"tall", "antonym":"short"},
]

# Next, we specify thhe template to format the examples we have provided.
# We usse the `promptTemplate` class for this
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""
# finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
  # These are the examples we want to inesrt into the prompt.
  examples=examples,
  # Thus us how we want to format the examples when we insert them into the prompt
  example_prompt = example_prompt,
  # The prefix is some text that goes before the examples in the prompt
  # usually, this is where the user input will go
  # The input variables are the variables that ahte overall prompt expects.
  prefix="Give the eantonym of every inpyt\n",
  # The example_seperator
  suffix="Word: {input}\n Antonym: ",

  input_variables=["input"],
  # The example seperator is the string we will use to join the prefix, examples,
  example_separator="\n",
)

print(few_shot_prompt.format(input='big'))

chain = LLMChain(llm=llm, prompt=few_shot_prompt)
chain.run("big")

NameError: name 'example_selector' is not defined

In [ ]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)
chain.run("big")

1. gemma api key -> test it
use gemma